In [1]:
import numpy as np
import openmc
import openmc.deplete as od
import re 
from openmc.data import zam, gnds_name

In [2]:
og_mats = openmc.Materials.from_xml(path='openmc_results/materials.xml')
geometry = openmc.Geometry.from_xml(path='openmc_results/geometry.xml', materials=og_mats)
settings = openmc.Settings.from_xml(path='openmc_results/settings.xml')

model = openmc.Model(materials=og_mats, geometry=geometry, settings=settings)
op = od.CoupledOperator(model, chain_file='openmc_results/chain_endfb71_ace.xml')

In [9]:
op._burnable_nucs

['H1',
 'H2',
 'H3',
 'He3',
 'He4',
 'Li6',
 'Li7',
 'Be9',
 'B10',
 'B11',
 'N14',
 'N15',
 'O16',
 'O17',
 'F19',
 'Na22',
 'Na23',
 'Mg24',
 'Mg25',
 'Mg26',
 'Al27',
 'Si28',
 'Si29',
 'Si30',
 'P31',
 'S32',
 'S33',
 'S34',
 'S36',
 'Cl35',
 'Cl37',
 'Ar36',
 'Ar38',
 'Ar40',
 'K39',
 'K40',
 'K41',
 'Ca40',
 'Ca42',
 'Ca43',
 'Ca44',
 'Ca46',
 'Ca48',
 'Sc45',
 'Ti46',
 'Ti47',
 'Ti48',
 'Ti49',
 'Ti50',
 'V50',
 'V51',
 'Cr50',
 'Cr52',
 'Cr53',
 'Cr54',
 'Mn55',
 'Fe54',
 'Fe56',
 'Fe57',
 'Fe58',
 'Co58',
 'Co58_m1',
 'Co59',
 'Ni58',
 'Ni59',
 'Ni60',
 'Ni61',
 'Ni62',
 'Ni64',
 'Cu63',
 'Cu65',
 'Zn64',
 'Zn65',
 'Zn66',
 'Zn67',
 'Zn68',
 'Zn70',
 'Ga69',
 'Ga71',
 'Ge70',
 'Ge72',
 'Ge73',
 'Ge74',
 'Ge76',
 'As74',
 'As75',
 'Se74',
 'Se76',
 'Se77',
 'Se78',
 'Se79',
 'Se80',
 'Se82',
 'Br79',
 'Br81',
 'Kr78',
 'Kr80',
 'Kr82',
 'Kr83',
 'Kr84',
 'Kr85',
 'Kr86',
 'Rb85',
 'Rb86',
 'Rb87',
 'Sr84',
 'Sr86',
 'Sr87',
 'Sr88',
 'Sr89',
 'Sr90',
 'Y89',
 'Y90',
 'Y91',
 '

In [22]:
fuel = og_mats[0]
percents = []
nucs = []
at_mass = []
for nuc, pt, tp in fuel.nuclides:
    nucs.append(nuc)
    percents.append(pt)

percents = np.array(percents)
serp_spec = []
mass_percents_dict = dict(zip(nucs, percents))
burnable_nucs = op._burnable_nucs
decay_nucs = op._decay_nucs
all_nucs = burnable_nucs# + decay_nucs
all_nucs.sort(key=lambda x: list(op.number.nuclides).index(x))

In [32]:
serp_spec = []
for nuc in all_nucs:
    z, a, m = zam(nuc)
    serp_zam = str(z*1000 + a + (300*int(bool(m)) + 100*m))
    #if nuc in burnable_nucs:
    #    serp_zam = ''.join([serp_zam, '.82c'])
    #else:
    #    serp_zam = ''.join([serp_zam, str(m)])
    serp_zam = ''.join([serp_zam, '.82c'])
    serp_space = ''.join([" "]*(20 - len(serp_zam)))
    serp_zam = serp_space + serp_zam
    if nuc in mass_percents_dict:
        mass_percent = mass_percents_dict[nuc]
    else:
        mass_percent = 0.0
    serp_spec.append(serp_zam + "  -" + str(mass_percent) + '\n')

In [33]:
serp_spec

['            1001.82c  -0.0\n',
 '            1002.82c  -0.0\n',
 '            1003.82c  -0.0\n',
 '               10040  -0.0\n',
 '               10050  -0.0\n',
 '               10060  -0.0\n',
 '               10070  -0.0\n',
 '            2003.82c  -0.0\n',
 '            2004.82c  -0.0\n',
 '               20050  -0.0\n',
 '               20060  -0.0\n',
 '               20070  -0.0\n',
 '               20080  -0.0\n',
 '               20090  -0.0\n',
 '               20100  -0.0\n',
 '               30040  -0.0\n',
 '               30050  -0.0\n',
 '            3006.82c  -3.937496230427713e-06\n',
 '            3007.82c  -0.07874598711233247\n',
 '               30080  -0.0\n',
 '               30090  -0.0\n',
 '               30100  -0.0\n',
 '               30110  -0.0\n',
 '               30120  -0.0\n',
 '               40050  -0.0\n',
 '               40060  -0.0\n',
 '               40080  -0.0\n',
 '            4009.82c  -0.02255755625117608\n',
 '               40100  -0

In [14]:
with open('serp_fuel_spec.txt', 'w') as f:
    f.writelines(serp_spec)

In [6]:
serp_spec = []
for name, tup in og_mats[2].get_nuclide_densities().items():
    name, mass_percent, percent_type = tup
    z, a, m = zam(name)
    serp_zam = str(z*1000 + a + (300*int(bool(m)) + 100*m))
    serp_zam = ''.join([serp_zam, '.82c'])
    serp_space = ''.join([" "]*(20 - len(serp_zam)))
    serp_zam = serp_space + serp_zam
    serp_spec.append(serp_zam + "  -" + str(mass_percent) + '\n')

In [7]:
serp_spec

['           42092.82c  -0.01683826965647346\n',
 '           42094.82c  -0.010789582056756033\n',
 '           42095.82c  -0.018840557266836578\n',
 '           42096.82c  -0.01999840362163272\n',
 '           42097.82c  -0.011613114609036522\n',
 '           42098.82c  -0.02974484682761688\n',
 '           42100.82c  -0.012175225961647808\n',
 '           24050.82c  -0.0029215806359028196\n',
 '           24052.82c  -0.058589553645942136\n',
 '           24053.82c  -0.006771510773072629\n',
 '           24054.82c  -0.00171735494508243\n',
 '           26054.82c  -0.0016936674668120565\n',
 '           26056.82c  -0.027570458633868738\n',
 '           26057.82c  -0.0006481105841023909\n',
 '           26058.82c  -8.776331521681765e-05\n',
 '            6000.82c  -0.0006\n',
 '           25055.82c  -0.0035\n',
 '           14028.82c  -0.0009187351728283676\n',
 '           14029.82c  -4.831750292786396e-05\n',
 '           14030.82c  -3.2947324243768426e-05\n',
 '           74180.82c  

In [8]:
with open('serp_hast_spec.txt', 'w') as f:
    f.writelines(serp_spec)